# The Agents of FloAI

This notebook shows different kinds of agents that can be built using flo-ai. These agents have their own specialities, and this notebook tries to explain them.

Here are the major types of agents:

1. Agentic Agents (`kind: agentic`): Any agent created in flo-ai by default tries to be an agentic agent. This agent always needs a tool, meaning an agent without a tool if specified as agentic agent whill throw an exception

2. LLM Agents (`kind: llm`): These agents are simply an LLM which can answer any questions asked to it. The agents dont except tools. If tool is passed to an agent of type llm, they are ignored.

3. Tool LLM (`kind: tool`): These agents are just tools or functions that can be executed on the current state. Within the tool will be given the current state as the input, meaning the history of what happened in the flo until now

In [1]:
from flo_ai import Flo
from flo_ai import FloSession
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from dotenv import load_dotenv
load_dotenv()

True

## Setup
Create the Flo session, setup tools

In [2]:
from langchain_community.tools.tavily_search.tool import TavilySearchResults

llm = ChatOpenAI(temperature=0, model_name='gpt-4o-mini')
session = FloSession(llm)

session.register_tool(
    name="InternetSearchTool", 
    tool=TavilySearchResults()
)

## Agentic Agent (agentic)

Here we are gonna create a simple weather assitant flo agent that can check the whether by an internet searching tool.

As you can see the kind is `agentic`

In [8]:
simple_weather_checking_agent = """
apiVersion: flo/alpha-v1
kind: FloAgent
name: weather-assistant
agent:
    name: Weather Assistant
    kind: agentic
    job: >
      Given the city name you are capable of answering the latest whether this time of the year by searching the internet
    tools:
      - name: InternetSearchTool
"""

In [9]:
flo = Flo.build(session, simple_weather_checking_agent)

print(flo.invoke("Whats the whether in california"))



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'California weather October 2023'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'California City', 'region': 'California', 'country': 'United States of America', 'lat': 35.13, 'lon': -117.99, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1725364509, 'localtime': '2024-09-03 04:55'}, 'current': {'last_updated_epoch': 1725363900, 'last_updated': '2024-09-03 04:45', 'temp_c': 23.1, 'temp_f': 73.6, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 10, 'wind_dir': 'N', 'pressure_mb': 1016.0, 'pressure_in': 30.0, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 31, 'cloud': 0, 'feelslike_c': 23.8, 'feelslike_f': 74.9, 'windchill_c': 22.7, 'windchill_f': 72.9, 'heatindex_c': 23.7, 'heatindex_f': 74.7, 'dewpoint_c': 5.0, 'dewpoint_f': 41.0, 'vis_km

## LLM Agent (agentic)

Here we are gonna create a simple llm math assitant flo agent that can check answer any math question

As you can see the kind is `llm`

In [10]:
simple_llm_agent = """
apiVersion: flo/alpha-v1
kind: FloAgent
name: llm-assistant
agent:
    name: Ask llm anything
    kind: llm
    job: >
      You are a high school maths teacher. Answer any questions the students ask 
"""

In [11]:
flo = Flo.build(session, simple_llm_agent)

print(flo.invoke("What is pythagorus theorum"))

Pythagoras' Theorem is a fundamental principle in geometry that relates to right-angled triangles. It states that in a right triangle, the square of the length of the hypotenuse (the side opposite the right angle) is equal to the sum of the squares of the lengths of the other two sides. 

The theorem can be expressed with the formula:

\[ c^2 = a^2 + b^2 \]

where:
- \( c \) is the length of the hypotenuse,
- \( a \) and \( b \) are the lengths of the other two sides.

This theorem is useful for calculating the length of one side of a right triangle if the lengths of the other two sides are known. Would you like to see an example of how to use it?


#

In [15]:
from typing import Optional, Type
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

class PrintStateTool(BaseTool):
    name = "printStateTool"
    description = "Just print the state"

    def _run(
        self, **kwargs
    ) -> str:
        print(kwargs)
        return "Tool call success"
    
session.register_tool(
    name="printStateTool", 
    tool=PrintStateTool()
)
    
simple_tool_agent = """
apiVersion: flo/alpha-v1
kind: FloAgent
name: llm-assistant
agent:
    name: tool-to-print-state
    kind: tool
    tools:
        - name: printStateTool
"""

flo = Flo.build(session, simple_tool_agent)

print(flo.invoke("Print what I am saying"))

{'messages': [HumanMessage(content='Print what I am saying')]}
Tool call success
